In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
#from bs4 import BeautifulSoup
from collections import defaultdict
#import requests
%matplotlib inline
from sklearn.svm import SVC
from nltk.stem.snowball import SnowballStemmer

In [2]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

from tensorflow.keras.preprocessing import sequence
# fix random seed for reproducibility
np.random.seed(7)

In [3]:
# load the dataset but only keep the top n words, zero the rest
# only take 5000 docs
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

17465344/17464789 [==============================] - 0s 0us/step


In [4]:
# truncate and pad input sequences
# only take maxiumn 500 word
max_review_length = 500

# if < 500 words, add 0 in front
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

# n: 25000
# maxiumn words 500
print(X_train.shape)
print(X_test.shape)

(25000, 500)
(25000, 500)


In [17]:
## transform into n x 32 dimensions as output of embeding layer and input for next layer
embedding_vecor_length = 32
model = Sequential()

## input dim
#embedding = Embedding(top_words, embedding_vecor_length, input_length=max_review_length)
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
LSTM = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=64,epochs=3)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/3
  896/25000 [>.............................] - ETA: 3:53 - loss: 0.6934 - acc: 0.4520

KeyboardInterrupt: 

In [7]:
score = model.evaluate(X_test,y_test)

25000/25000 [==============================] - 70s 3ms/step


In [8]:
score

[0.3035099205780029, 0.87812]

In [16]:
from sklearn.metrics import accuracy_score, confusion_matrix
y_test_pred = model.predict_classes(X_test)
accuracy_score(y_test,y_test_pred)

0.87812